<a href="https://colab.research.google.com/github/rrgiyer/PatternPy/blob/main/Copy_of_continuous_straddle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: get Option chain from NSE for both nifty and BN for present expiry, next week expiry and week after next expiry.

!pip install nsepy
!pip install nsetools
!pip install nsepython

# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

from nsepython import nse_optionchain_scrapper

def get_weekly_expiries():
    # Fetch Nifty option chain data
    nifty_data = nse_optionchain_scrapper("NIFTY")

    # Fetch Bank Nifty option chain data
    banknifty_data = nse_optionchain_scrapper("BANKNIFTY")

    # Extract expiry dates from the response
    nifty_expiries = nifty_data['records']['expiryDates']
    banknifty_expiries = banknifty_data['records']['expiryDates']

    return {
        'NIFTY': nifty_expiries,
        'BANKNIFTY': banknifty_expiries
    }

# Example usage:
expiries = get_weekly_expiries()
print("NIFTY Expiries:", expiries['NIFTY'])
print("BANKNIFTY Expiries:", expiries['BANKNIFTY'])



def get_option_chain_for_expiries(symbol):
    # Fetch option chain data for the symbol (NIFTY or BANKNIFTY)
    option_chain_data = nse_optionchain_scrapper(symbol)

    # Get the first three expiries (current, next week, and week after)
    expiries = option_chain_data['records']['expiryDates'][:3]

    # Dictionary to store option chain for each expiry
    option_chain_by_expiry = {}

    for expiry in expiries:
        # Filter options by expiry date
        filtered_data = [
            option for option in option_chain_data['records']['data']
            if option['expiryDate'] == expiry
        ]

        # Extract CE and PE data including implied volatility
        options_with_iv = []
        for option in filtered_data:
            ce_ltp = option.get('CE', {}).get('lastPrice', 'N/A')
            pe_ltp = option.get('PE', {}).get('lastPrice', 'N/A')
            ce_iv = option.get('CE', {}).get('impliedVolatility', 'N/A')
            pe_iv = option.get('PE', {}).get('impliedVolatility', 'N/A')

            options_with_iv.append({
                'strikePrice': option['strikePrice'],
                'CE LTP': ce_ltp,
                'PE LTP': pe_ltp,
                'CE IV': ce_iv,
                'PE IV': pe_iv,
            })

        option_chain_by_expiry[expiry] = options_with_iv

    return option_chain_by_expiry

# Example usage
nifty_option_chain = get_option_chain_for_expiries('NIFTY')
for expiry, options in nifty_option_chain.items():
    print(f"Expiry: {expiry}")
    for option in options:
        print(f"Strike Price: {option['strikePrice']}, CE LTP: {option['CE LTP']}, PE LTP: {option['PE LTP']}, CE IV: {option['CE IV']}, PE IV: {option['PE IV']}")

banknifty_option_chain = get_option_chain_for_expiries('BANKNIFTY')
for expiry, options in banknifty_option_chain.items():
    print(f"Expiry: {expiry}")
    for option in options:
        print(f"Strike Price: {option['strikePrice']}, CE LTP: {option['CE LTP']}, PE LTP: {option['PE LTP']}, CE IV: {option['CE IV']}, PE IV: {option['PE IV']}")





NIFTY Expiries: ['26-Sep-2024', '03-Oct-2024', '10-Oct-2024', '17-Oct-2024', '24-Oct-2024', '31-Oct-2024', '28-Nov-2024', '26-Dec-2024', '27-Mar-2025', '26-Jun-2025', '24-Dec-2025', '25-Jun-2026', '31-Dec-2026', '24-Jun-2027', '30-Dec-2027', '29-Jun-2028', '28-Dec-2028', '28-Jun-2029']
BANKNIFTY Expiries: ['25-Sep-2024', '01-Oct-2024', '09-Oct-2024', '16-Oct-2024', '23-Oct-2024', '30-Oct-2024', '27-Nov-2024', '24-Dec-2024', '26-Mar-2025', '25-Jun-2025']
Expiry: 26-Sep-2024
Strike Price: 16000, CE LTP: 9788.2, PE LTP: 1.2, CE IV: 0, PE IV: 124.25
Strike Price: 17000, CE LTP: 8263.95, PE LTP: 1.05, CE IV: 0, PE IV: 108.26
Strike Price: 18000, CE LTP: 7749, PE LTP: 1.3, CE IV: 0, PE IV: 96.22
Strike Price: 19000, CE LTP: 6768.8, PE LTP: 1.5, CE IV: 0, PE IV: 83.92
Strike Price: 20000, CE LTP: 5807.45, PE LTP: 4.45, CE IV: 0, PE IV: 79.45
Strike Price: 21000, CE LTP: 4788.1, PE LTP: 2.5, CE IV: 0, PE IV: 61.61
Strike Price: 22000, CE LTP: 3798.95, PE LTP: 3.7, CE IV: 0, PE IV: 51.22
Strike

In [ ]:
# prompt: module to calculate greeks basis BS model, with int free rate of 10% and run this code every 1 minute

!pip install py_vollib

import numpy as np
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta, rho
import time
import datetime

# Assuming you have your option chain data in 'option_chain'
# Replace this with your actual option chain data
# Example Option Chain Structure
option_chain = {
    "NIFTY": {
        "CE": [
            {"strike_price": 19000, "ltp": 100, "implied_volatility": 0.2},
            {"strike_price": 19100, "ltp": 80, "implied_volatility": 0.18},
        ],
        "PE": [
            {"strike_price": 19000, "ltp": 120, "implied_volatility": 0.22},
            {"strike_price": 18900, "ltp": 150, "implied_volatility": 0.25},
        ]
    }
}

def calculate_greeks(option_data, underlying_price, risk_free_rate=0.10):
  """Calculates greeks for given option data."""
  try:
      flag = 'c' if option_data['option_type'] == 'CE' else 'p'
      price = option_data['ltp']
      strike_price = option_data['strike_price']
      time_to_expiry = (expiry_date - datetime.datetime.now()).days / 365.0
      volatility = option_data['implied_volatility']

      bs_price = bs(flag, underlying_price, strike_price, time_to_expiry, risk_free_rate, volatility)
      calculated_delta = delta(flag, underlying_price, strike_price, time_to_expiry, risk_free_rate, volatility)
      calculated_gamma = gamma(flag, underlying_price, strike_price, time_to_expiry, risk_free_rate, volatility)
      calculated_vega = vega(flag, underlying_price, strike_price, time_to_expiry, risk_free_rate, volatility)
      calculated_theta = theta(flag, underlying_price, strike_price, time_to_expiry, risk_free_rate, volatility)
      calculated_rho = rho(flag, underlying_price, strike_price, time_to_expiry, risk_free_rate, volatility)

      return {
          "delta": calculated_delta,
          "gamma": calculated_gamma,
          "vega": calculated_vega,
          "theta": calculated_theta,
          "rho": calculated_rho,
      }
  except Exception as e:
      print(f"Error in calculation: {e}")
      return None

# Example Usage
underlying_price = 19000
expiry_date = datetime.datetime(2023, 11, 20)

if option_chain:
    for symbol, symbol_data in option_chain.items():
        for option_type, options in symbol_data.items():
            for option_data in options:
                option_data['option_type'] = option_type
                greeks = calculate_greeks(option_data, underlying_price, risk_free_rate=0.10)
                if greeks:
                    print(f"Option data: {option_data}")
                    print(f"Greeks: {greeks}")

while True:
    # Get current time
    current_time = datetime.datetime.now()
    print(f"Running at: {current_time}")

    # Here you would replace this with your actual code for retrieving option chain
    # Option chain data here should be updated for the current minute
    # option_chain = get_updated_option_chain()

    # Recalculate Greeks using the newly updated option chain data
    # ...

    time.sleep(60)


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 4.8 MB/s eta 0:00:00
  Created wheel for py_vollib: filename=py_vollib-1.0.1-py3-none-any.whl size=62831 sha256=3b0f97514b20a5fd89f490b87a026961437bf563c95148cc16d4f1158e9253e2
  Stored in directory: /root/.cache/pip/wheels/84/0c/fc/b68506eda40cccaeb0013be550ba904d253ec90eae2b156052
  Created wheel for py_lets_be_rational: filename=py_lets_be_rational-1.0.1-py3-none-any.whl size=24451 sha256=a6d58333ddcfc17ebc1c4cb7cf0082dca3c25c225d67c729769d5b566b0c2de9
  Stored in directory: /root/.cache/pip/wheels/32/d9/4f/360d4b0b9ec7c65583ec7c8e72fd2158c97ad04fbcb932891c
Successfully built py_vollib py_lets_be_rational


/usr/local/lib/python3.10/dist-packages/py_vollib/ref_python/black_scholes/__init__.py:83: RuntimeWarning: invalid value encountered in sqrt
  denominator = sigma * numpy.sqrt(t)
/usr/local/lib/python3.10/dist-packages/py_vollib/black_scholes/greeks/analytical.py:190: RuntimeWarning: invalid value encountered in sqrt
  return pdf(d_1)/(S*sigma*numpy.sqrt(t))
/usr/local/lib/python3.10/dist-packages/py_vollib/black_scholes/greeks/analytical.py:229: RuntimeWarning: invalid value encountered in sqrt
  return S * pdf(d_1) * numpy.sqrt(t) * 0.01
/usr/local/lib/python3.10/dist-packages/py_vollib/black_scholes/greeks/analytical.py:139: RuntimeWarning: invalid value encountered in sqrt
  two_sqrt_t = 2 * numpy.sqrt(t)
/usr/local/lib/python3.10/dist-packages/py_vollib/ref_python/black_scholes/__init__.py:118: RuntimeWarning: invalid value encountered in sqrt
  return d1(S, K, t, r, sigma) - sigma * numpy.sqrt(t)


Option data: {'strike_price': 19000, 'ltp': 100, 'implied_volatility': 0.2, 'option_type': 'CE'}
Greeks: {'delta': nan, 'gamma': nan, 'vega': nan, 'theta': nan, 'rho': nan}
Option data: {'strike_price': 19100, 'ltp': 80, 'implied_volatility': 0.18, 'option_type': 'CE'}
Greeks: {'delta': nan, 'gamma': nan, 'vega': nan, 'theta': nan, 'rho': nan}
Option data: {'strike_price': 19000, 'ltp': 120, 'implied_volatility': 0.22, 'option_type': 'PE'}
Greeks: {'delta': nan, 'gamma': nan, 'vega': nan, 'theta': nan, 'rho': nan}
Option data: {'strike_price': 18900, 'ltp': 150, 'implied_volatility': 0.25, 'option_type': 'PE'}
Greeks: {'delta': nan, 'gamma': nan, 'vega': nan, 'theta': nan, 'rho': nan}
Running at: 2024-09-22 03:02:42.765048
Running at: 2024-09-22 03:03:42.824372
Running at: 2024-09-22 03:04:42.877304
Running at: 2024-09-22 03:05:42.936267
Running at: 2024-09-22 03:06:42.995287
Running at: 2024-09-22 03:07:43.055251
Running at: 2024-09-22 03:08:43.115472
Running at: 2024-09-22 03:09:43.1

Short Strangle on nifty and BN

Strategy Premises:
the strategy envisages a continous adjusting condor with calander spread so as to take advantage of margin benefirs offered by calander offset and is positional in nature
target profit is 4% of margins deployed and stop loss @ 2% effectively targeting R:R of 1:2. if target exceeds 4%  profit is protected @ 4% and is trailed @1% /1% movement

Data required:

Option chain from NSE for both nifty and BN for present expiry,next expiry and week after next expiry. streaming tick data on all strikes appearing on the option chain, module to calculate greeks basis the streaming data (updated on a 60/30 sec basis basis)

Strategy Construct:

Instruments:
Sell position on both CE and PE side with 2 lots on week after next with a Delta nearest to 0.2
Buy 1 lot of next week CE/PE with delta less thanor equavalent to 0.1
This can be taken anytime between 15.20 and 15.25 @ prevailing maret price

Adjustments are done only on the sell positions and buy positions are left alone

Calculations: Break even points for the position to be calculated at entry and after each adjustment
Margin deployed to be checked to reassess the R:R

Adjustments:

When the underlying (BN/Nifty) moves by 1/6th of the range from the starting position, the sell position in profit is booked and fresh position sellequai to the delta of the O/s sell position is taken up.
  scenerios:
      a) only one of the leg is in green: exit the position as above
      b) both legs are in green/red: Exit both and re enter on both sides with Delta =.2
      
Break even points are rechecked for fresh limits and margins checked for revised R/R
This process is continued till either sl ot tsl is hit or time comes to 2 pmon the weekly expiry day at which point all positions are exited irrespective of the PNL.










In [ ]:
# prompt: Sell position on both CE and PE side with 2 lots on week after next with a Delta nearest to 0.2 Buy 1 lot of next week CE/PE with delta less thanor equavalent to 0.1 This can be taken anytime between 15.20 and 15.25 @ prevailing maret price
# Adjustments are done only on the sell positions and buy positions are left alone

import datetime

def find_nearest_delta_option(option_chain, target_delta, expiry_date):
  """Finds the option with the delta closest to the target delta."""
  best_option = None
  min_delta_diff = float('inf')

  for option_type, options in option_chain.items():
    for option_data in options:
      if option_data['expiry_date'] == expiry_date:
        greeks = calculate_greeks(option_data, underlying_price)  # Assuming underlying_price is defined
        if greeks and 'delta' in greeks:
          delta_diff = abs(greeks['delta'] - target_delta)
          if delta_diff < min_delta_diff:
            min_delta_diff = delta_diff
            best_option = option_data
  return best_option

# Assuming you have your option chain data in 'option_chain'
# Replace this with your actual option chain data
# Example Option Chain Structure (you'll need to update this to include expiry dates)
option_chain = {
    "NIFTY": {
        "CE": [
            {"strike_price": 19000, "ltp": 100, "implied_volatility": 0.2, "expiry_date": datetime.date(2023, 11, 20)},  # Example expiry date
            {"strike_price": 19100, "ltp": 80, "implied_volatility": 0.18, "expiry_date": datetime.date(2023, 11, 20)},
        ],
        "PE": [
            {"strike_price": 19000, "ltp": 120, "implied_volatility": 0.22, "expiry_date": datetime.date(2023, 11, 20)},
            {"strike_price": 18900, "ltp": 150, "implied_volatility": 0.25, "expiry_date": datetime.date(2023, 11, 20)},
        ]
    }
}

# Find the week after next expiry date (You'll need to implement logic to calculate this)
week_after_next_expiry = datetime.date(2023, 11, 27)  # Example

# Find the next week expiry date (You'll need to implement logic to calculate this)
next_week_expiry = datetime.date(2023, 11, 20)  # Example

# Sell 2 lots of CE and PE with delta nearest to 0.2 on week after next expiry
ce_sell_option = find_nearest_delta_option(option_chain['NIFTY'], 0.2, week_after_next_expiry)
pe_sell_option = find_nearest_delta_option(option_chain['NIFTY'], -0.2, week_after_next_expiry)

if ce_sell_option and pe_sell_option:
  print("Sell 2 lots of CE:", ce_sell_option)
  print("Sell 2 lots of PE:", pe_sell_option)

# Buy 1 lot of CE and PE with delta less than or equal to 0.1 on next week expiry
ce_buy_option = find_nearest_delta_option(option_chain['NIFTY'], 0.1, next_week_expiry)
pe_buy_option = find_nearest_delta_option(option_chain['NIFTY'], -0.1, next_week_expiry)

if ce_buy_option and pe_buy_option:
  print("Buy 1 lot of CE:", ce_buy_option)
  print("Buy 1 lot of PE:", pe_buy_option)


NameError: name 'calculate_greeks' is not defined